# Mini Tumor Droplet Classification

The purpose of this notebook is to offer an easy to understand format for working with the code. Moreover, we also tried to make this notebook in such a way that any modificaitons to it (e.g changing the training arguments or model hyperparameter) should be as easy and clear as possible. Therefore, we hope that this notebook will serve and an effective main interface to work with our code

However, before starting, we need to import the necessary libraries. These imports include both external libraries we made use of during the project but mainly consist of our own code.

In [ ]:
%load_ext autoreload
%autoreload 2

import json
from jsonschema import validate
from pathlib import Path

from pipeline.dataset.make_dataset import make_dataset, view_samples
from pipeline.training.train import train
from pipeline.models.predict import predict
from pipeline.models.check_predictions import check_predictions, view_sample_predictions
from pipeline.models.get_confidences import get_confidence

with open("pipeline/utils/schema_predict.json") as f:
    schema_predict = json.load(f)

## Introduction

Currently, the repository has three main functionalities: data preprocessing, model training and prediction. Each of these functionalities are demonstrated in depth furhter down in the notebook in order to make it as easy as possible to use.

Although all the different functionalities depend on each other, the pre-processing of the raw-data (brightfield images of collection of tumor droplets) is the most crucial part and is also the foundation for the other steps. 

## 1. Preprocess Data

### 1.1 The arguments file

Below we create two dictionaries, `dataset_train_args` and `dataset_test_args`. These two dictionaries contain the parameters for creating the train and test dataset respectivley and they are sued for training and testing a mdoel respectivley. For each key (what comes before the colon), the comment on the right describes its funcitonality. `dataset_args` defines arguments which are shared between `dataset_train_args` and `dataset_test_args` (most arguments), but the dataset directories, image directories and (for testing) histogram directories are defined separately below, as these should not be the same for the training and testing datasets. 


In [ ]:
# Common settings used for both training and test dataset
dataset_args = {
    "scoring_dir": "data/ColonCancer/ScoringFiles",         # Location of scoring files
    "temp_dir": "data/ColonCancer/PreprocessingFiles",      # Location for temporary storage during pre-processing
    "channels": None,                                       # Channel extracted from the image
    "fg_threshold": 0.15,                                   # Threshold parameter in foreground extraction
    "fg_theta": 100,                                        # Number of angular discretization points in foreground extraction
    "fg_r": 1000,                                           # Number of radial discretization points in foreground extraction
    "filters": [],                                          # Filtering methods for files
    "validators": ["NonEmpty"],                             # Validation methods for droplets - known outliers
    "detectors": ["IsolationForest","LocalFactor"],         # Outlier detection methods for droplets - learned outliers
    "extract_labels": True,                                 # Extract labels, needs to be false if labels are unknown
    "overwrite": True,                                      # Overwrite existing datasets
}

# Specifications specific for training and test dataset
# Need to specify different locations
dataset_train_args = {
  "dataset_dir": "data/ColonCancer/TrainDataset",           # Location of training images
  "image_dir":"data/ColonCancer/TrainImages/",              # Location of training dataset
}
dataset_train_args.update(dataset_args)

dataset_test_args = {
  "dataset_dir": "data/ColonCancer/TestDataset",            # Location of test images
  "image_dir": "data/ColonCancer/TestImages/",              # Location of test dataset
  "hist_dir": dataset_train_args["dataset_dir"],            # Use the same normalization for training and testing
}
dataset_test_args.update(dataset_args)


If you just want to predict on data where you have no labels, you have to create a inference dataset called `dataset_inf`.

In [ ]:
dataset_inf_args = {
  "dataset_dir": "data/ColonCancer/InfDataset",             # Location of inference images
  "image_dir": "data/ColonCancer/TestImages/",              # Location of inference dataset
  "hist_dir": dataset_train_args["dataset_dir"],            # Use the same normalization for training and inference
}
dataset_inf_args.update(dataset_args)
dataset_inf_args["extract_labels"]=False

### 1.2 Preprocess the data

To preprocess the data with the arguments specified above, run the following cell. 

The dataset will be saved in at the corresponding file, as determinded in the arguments files above. Additionally, in the same folder, there will be a ```meta_data.json``` that contains some meta data about the dataset, in particular, the parameters with which it was created. This allows to recreate the dataset and this can be very useful for recreating an experiment or for knowing how a dataset was created.

In [ ]:
print("Creaing training dataset...")
make_dataset(**dataset_train_args)

In [ ]:
print("Creaing testing dataset...")
make_dataset(**dataset_test_args)

Again, if you want to predict on data without labels, do the following.

In [ ]:
print("Creaing inference dataset...")
make_dataset(**dataset_inf_args)

To get some understanding for how the pre-processed droplets look like, you can run the following cell. The figure displays different droplets and their corresponding label. It's these droplets and labels and are used to train our deep learning methods.

In [ ]:
num_views =2 # number of samples you want to be plotted.
view_samples(num_views, Path(dataset_train_args["dataset_dir"]))
view_samples(num_views, Path(dataset_test_args["dataset_dir"]))

## 2. Training a Predictor on Labeled Data

If you aquire new data together with labels from the microscope, for which you want to train a new model, you can make use of the code in the below section. This allows you to easily train new models, both for new types of data but also for the existing data with modified parameters.

### 2.1 The Argument File

The argument file ```training_args``` is where you can specifiy the parameters you want to use during training. You should keep the JSON structure of the file, and adapt the parameters according to the description on the right hand side below. Our understanding is that these parameters should be optimal or at least very close to being optimal. However, there can still be worth exploring different parameters further to increase the understanding and for new data, it could be that the optimal parameters change slightly. This will of course depend on the exact characteristics of the new data. Most likley, the optiaml parameter wont change much.

In [ ]:
# Specifications specific for training
training_args = {
    "dataset_dir": "data/ColonCancer/TrainDataset/",        # Location of training dataset
    "model_dir": "models/",                                 # Location to store model
    "model_name": "my-new-model",                           # Name to store model under
    "pretrained_name": "timm-resnet34",                     # Name of pre-trained base model
    "freeze": False,                                        # Freeze parts of model
    "batch_size": 50,                                       # Training batch size
    "epochs": 10,                                           # Number of epochs
    "learning_rate": 1e-4,                                  # Learning rate used by optimizer
    "validation_split": 0.05,                               # Fraction of data used for validation
    "seed": 42,                                             # Random number seed
    "validate_every": 150,                                  # How often validation is performed
    "predict_dead" : True,                                  # Predict binary value if all cells are dead or not (rather than e.g. 4 classes)
    "wb": False,                                            # Use weights and biases
    "image_transforms" : [                                  # Transfromations applied to the images for augmentation
        "RandomHorizontalFlip", 
        "RandomVerticalFlip", 
        "RandomRotation", 
        "ColorJitter"
    ]
}

### 2.2 Training

To train a model using the arguments specified in ```training_args```, run the following cell.

To watch training take place, you can pass wb the parameter ```true```. This will automatically open a browser tab to [weights and biases](https://wandb.ai/site) where you can watch your model train. This is especially if you would train a model remote (e.g the Euler cluster) so that you can easily monitor the progress, esomething especially useful for bigger experiments.

The model together with the parameters specified above used to train it, as well as with some key statistics will be saved in results_directory, within the folder that has the model_name plus date and time of training. For example, in output_directory, you could find a subfolder

```my-new-model-2022-11-29-11-31```.

In [ ]:
train(**training_args)

### 3. Predicting on New Data Using a Trained Predictor

Once you have trained a model, you can use it to predict on new data, either with labels to test you model, or without labels for inference.

#### 3.1 The Argument File

The arguments file ```args_predict``` is where you define all arguments for the predictions you want to make. Most importantly, you need to choose whether you want our test dataset or you inference dataset. 

1. input_dir: The directory in which the new data, which you want to predict on, is stored, as well as where new folders with datasets etc. are created.
1. dataset_name: Since in order to predict on the data, you need to first need to create a new dataset that has undergone preprocessing, this will be the first step. dataset_name will be the name of this dataset.
1. extract_labels: Here you specify whether you have labels (True) or or not (False)
1. pred_name: The name you want to give the predictions.
1. model_dir: directory where the model and arguments files which you want to use is stored.
1. model_name: The name of the model you want to predict with.
1. predict_dead: whether to only make binary predictions dead/alive

Generally, if a folder or file with an identical name already exists *it will be overwritten!*

In [ ]:
predict_data_path = dataset_inf_args["dataset_dir"]
# predict_data_path = dataset_test_args["dataset_dir"]
args_predict = {
    "input_dir": '/'.join(predict_data_path.split('/')[:-1]) + '/',
    "dataset_name": predict_data_path.split('/')[-1],
    "pred_name": "predictions",
    "model_dir": "models/final_model/", 
    "model_name": "binary_model", 
    "predict_dead": training_args['predict_dead'] 
}
try: 
    validate(instance=args_predict, schema=schema_predict)
except: 
    raise RuntimeError("the arguments file is not valid")

### 3.2 Predicting

We provide models that we have fine tuned to the best of our capabilities, that you can use on your new data. But of course, you can also use you own new models. How well our models perform will depend a lot on how similar/disimlar the droplets you want to predict are to the ones we trained on.

To be able to predict on new data, it (the new data) must first be preprocessed in the same manner as the dataset on which the model was trained (this is where the meta data file comes in very handy if it's an old model). This was demonstrated in the beginning of the notebook and the resulting dataset should be saved in `dataset_test_args["dataset_dir"]`. Alternatively you can specify the location using `args_predict["input_dir"]` and `args_predict["dataset_name"]` above. 

To predict on this dataset, just run the following cell. The predictions will be saved in the same folder as the dataset, with the name you gave the predictions. The predictions file also contains the information which model etc. was used, so you can replicate your results.

In [ ]:
predict(args_predict)

You can use the following cell if you use your test dataset, which has labels. You can check how well the model did by running the following cell. The figure shows the confusion matrix of the predictions and is very useful for analyzing and reasoing about the behaviour of the model.

In [ ]:
check_predictions(args_predict)

### Confidences
The cell below lets you compute confidence intervals on the predictions for this drug. Refer to the report for a discussion on the number of required samples. Importantly, you should be using your inference dataset now (but you could also use the test dataset to check functionality).

In [ ]:
alpha = .1
method= None # None is more pessimistic than method='deterministic'
pred_dir = args_predict["input_dir"] + args_predict["dataset_name"] + '/' + args_predict["pred_name"]+".json"

confidences = get_confidence(
    pred_dir=pred_dir,
    confusion_dir=args_predict["model_dir"] + 'confusion.pickle', alpha=alpha, method=method, 
)

if isinstance(confidences, tuple): confidences, plot = confidences

for d, res in confidences.items():
    print(f'\t\tResults for drug {d}')
    for i, c in enumerate(res['mean']):
        print(f'mean occurence of label {i}: ', c)

    for i, c in enumerate(res['ci']):
        print(f'confidence interval for occurence of label {i}: ', c)
    print('')

To view some sample droplets together with their _predictions_ (not labels), run the following cell. This might be helpful as a sanity check. The distinction between predictions and labels is very important since it's possible for a badly trained model to make predictions that are completly wrong.

In [ ]:
num_views=3
view_sample_predictions(num_views, args_predict)